In [1]:
from __future__ import division
import math

## Review of basics of Hypothesis and Inference - From Grus' book (with small changes)

The binomial distribution arises as the sum of bernoulli random variables. Using the central limit theorem, we know that this sum is a random variable with distribution approaching the normal distribution, mean = n*p and variance = n*p*(1-p)

In [2]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

Other basic concepts covered are the CDF (cummulative density function) which determines P(X < x), the probability that the random variable X will be below x. Other definitions can be used:

In [5]:
from scipy.stats import norm

def normal_cdf(hi, mu=0, sigma=1):
    return norm.cdf(hi, loc = mu, scale = sigma)

# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)
    
# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also find inverse of CDF, which gives us ranges of values for a random variable X, 
for a given probability:

In [6]:
def inverse_normal_cdf(prob, mu, sigma):
    return norm.ppf(prob , loc= mu, scale=sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)
    
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds 
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

These definitions are useful for hypothesis testing, as we will see... Confidence intervals are strongly related with probability intervals (particularly when discussing the significance of a test). 

Let's run a simple hypothesis test. Let's say we want to determine wheter a coin is fair or not. 

Our null hypothesis should be Ho : p = 0.5. The alternative hypothesis is that p != 0.5. Let's say that we run the experiment 1000 times. 

We accept the null hypothesis as valid, and then determine how willing are we to have a type I error, i.e., a false positive. This determines the SIGNIFICANCE of a test. For a 5% of significance, in 1000 flips we have:

In [7]:
#Accept Ho as valid, find the parameters for the normal that approaches a binomial:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [8]:
#Find the lower and upper bounds for a significance of 5% on a 1000-flip experiment:
print normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.0102483847719, 530.9897516152281)


This means, that if we carry on the experiment and we find a number of fips within this window, we accept Ho as valid with 5% significance.

The significance of a test is just part of the story. We also have the POWER of a test, which is the one minus the probability of having a false negative (fail to reject Ho when Ho is false).

Let's say that p =0.55. What is the power of the test in this case?

In [9]:
#Accept Ho as valid, find the parameters for the normal that approaches a binomial:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

#But, Ho is false, so find the parameters for the normal that approaches a binomial:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

#What is the probabilty of falling within the Ho acceptance interval, even though Ho is not valid?
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability
print power

0.886547781098


A more powerful test is a ONE-SIDED test. Instead of testing whether p!=0.5, we can test whether the coin is biased towards more heads (p>0.5). This increases the power of the test, because the 5% tail involved in the calculation is only one sided, so the upper bound for rejection is lower:

In [15]:
norm.ppf(0.95, loc=mu_0, scale=sigma_0) #Number of heads for rejection of Ho

526.00741939377792

And the power of the one sided test is (prob of accepting Ho even when Ha is true):

In [24]:
print 1-normal_probability_below(526, mu_1, sigma_1)

0.936437785529


An alternative (and more commonly used) approach to hypothesis testing is to find the probability of finding a value at least as extreme as the observation. Then, this probability is compared with the significance of the test for rejection/acceptance of the null hypothesis:

In [25]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)   

So, for 530 heads we have:

In [26]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

We don't reject the null hypothesis.

### Confidence intervals

An alternative approach to statistical inference is using confidence intervals. We estimate a parameter of the population and give an interval around the parameter.

For example, let's assume that we get 525 heads from 1000 flips, so that we estimate p=0.525.

Per central limit theorem we obtain the confidence interval for the p parameter:

In [29]:
print normal_two_sided_bounds(0.95, 0.525, math.sqrt(0.525*(1-0.525)/1000))

(0.49404900981534522, 0.55595099018465477)


Note that the confidence interval says that if you were to repeat the experiment 100 times, 95 percent of the times your estimate for p will fall within this interval (the interval itself is different for each experiment).

### A/B testing

Let's suppose that we have 2 websites, site A and site B. We want to determine if an ad presented in both sites is more effective in one of the sites. This is a hypothesis testing, where we should test if the ad click rates coming from the sites are different. 

The null hypothesis is Ra=Rb, the alternative hypothesis is Ra!=Rb. We perform the experiment 1000 times per site. This is a tow sample test, where we should use the following statistical parameters:

In [31]:
##
#
# running an A/B test
#
##

def estimated_parameters(N, n):  #Estimated parameters for a site
    p = n / N   #Ad click rate
    sigma = math.sqrt(p * (1 - p) / N) #sigma
    return p, sigma

#Define test statistic:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2) #Variances added in quadrature

The null hypothesis is that p_B=p_A. Let's assume that after the experiment, we find n_a = 200 and n_b = 180:

In [34]:
two_sided_p_value(a_b_test_statistic(1000,200,1000,180))

0.25414197654223603

The probability of finding this difference is of 0.25. So we don't reject the null hypothesis. 

## Redoing this section with python modules

Let's try to reproduce everything, with only python modules, not to reinvent the wheel.

In [37]:
#stats module has statistical tests...and normal distribution
from scipy.stats import norm
from scipy import stats

We found previously that if we flip a coin 1000, a hypothesis test for fairness with 5% significance would find (469.0102483847719, 530.9897516152281) as the number of heads (or tails). 

In [47]:
#two tailed p-value for 530 flips:
print stats.ttest_1samp([1 if x < 531 else 0 for x in range(1000)], 0.5)
print stats.ttest_1samp([1 if x < 470 else 0 for x in range(1000)], 0.5)

Ttest_1sampResult(statistic=1.9634089034286304, pvalue=0.049875746922877176)
Ttest_1sampResult(statistic=-1.8998404716564716, pvalue=0.057741809969881409)


Notice the p values... 0.05 as previosly found.

In [50]:
#one tailed p-value:
print stats.ttest_1samp([1 if x < 526 else 0 for x in range(1000)], 0.5)

Ttest_1sampResult(statistic=1.6457885978381699, pvalue=0.10012184152590092)


A one tailed test can be reproduced from a symmetric 2 tailed test. Just divide the p value by 2.

For confidence intervals, we use norm.interval:

In [53]:
stats.norm.interval(0.95, loc=0.525, scale=math.sqrt(0.525*(1-0.525)/1000))

(0.49404900981534522, 0.55595099018465477)

Lastly, for a two sample t test, such as A/B testing of a website:

In [55]:
stats.ttest_ind([1 if x < 200 else 0 for x in range(1000)], [1 if x < 180 else 0 for x in range(1000)])

Ttest_indResult(statistic=1.1397761740438677, pvalue=0.2545161805191154)